<a href="https://colab.research.google.com/github/choons1k2/2022-Fall-Opensource-AI/blob/main/gender_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
f_path = '/content/heart.csv'

import pandas as pd
heart = pd.read_csv(f_path)


#OneHotEncoding
heart = pd.get_dummies(heart)
heart.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Index(['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak',
       'HeartDisease', 'Sex_F', 'Sex_M', 'ChestPainType_ASY',
       'ChestPainType_ATA', 'ChestPainType_NAP', 'ChestPainType_TA',
       'RestingECG_LVH', 'RestingECG_Normal', 'RestingECG_ST',
       'ExerciseAngina_N', 'ExerciseAngina_Y', 'ST_Slope_Down',
       'ST_Slope_Flat', 'ST_Slope_Up'],
      dtype='object')

In [63]:


for i in range(20):

  Male = heart[ heart['Sex_M'] == 1 ]
  Female = heart[ heart['Sex_M'] == 0 ]
  Male = Male.sample(frac=0.266)


  heartTest = pd.concat([Male, Female])
  print(f"{i+1}. 남성 표본: {len(Male)}명, 여성 표본: {len(Female)}명")
  Positive = heartTest[ heartTest['HeartDisease'] == 1 ]
  Negative = heartTest[ heartTest['HeartDisease'] == 0 ]

  mPositive = len(Positive[Positive['Sex_M'] == 1])
  fPositive = len(Positive[Positive['Sex_M'] == 0])

  mNegative = len(Negative[Negative['Sex_M'] == 1])
  fNegative = len(Negative[Negative['Sex_M'] == 0])

  mRate = mPositive / (mPositive + mNegative)
  fRate = Positive / (fPositive + fNegative)
  
  print(f"남성 양성률: {mRate}")
  print(f"여성 양성률: {fRate}\n")

1. 남성 표본: 193명, 여성 표본: 193명
남성 양성률: 0.6217616580310881
여성 양성률:           Age  RestingBP  Cholesterol  FastingBS     MaxHR   Oldpeak  \
513  0.321244   0.580311     1.336788   0.000000  0.777202  0.006736   
210  0.248705   0.549223     1.362694   0.005181  0.569948  0.000000   
528  0.253886   0.678756     0.735751   0.000000  0.658031  0.007772   
565  0.316062   0.730570     1.512953   0.000000  0.595855  0.008808   
343  0.269430   0.673575     0.000000   0.005181  0.621762  0.000000   
..        ...        ...          ...        ...       ...       ...   
897  0.284974   0.663212     1.062176   0.000000  0.673575  0.010363   
901  0.300518   0.880829     1.165803   0.005181  0.756477  0.014508   
909  0.326425   0.642487     1.020725   0.000000  0.704663  0.000000   
912  0.295337   0.725389     1.248705   0.000000  0.637306  0.001036   
916  0.295337   0.673575     1.222798   0.000000  0.901554  0.000000   

     HeartDisease     Sex_F     Sex_M  ChestPainType_ASY  ...  \
513    

남성 표본: 193명, 여성 표본: 193명
남성 양성 확률: 0.6373056994818653
여성 양성 확률: 0.25906735751295334


